With gradient factors, you interpolate between this line and the ambient line.

The ambient line is: $y=x$.

If we use $GF=80%$, we just draw a vertical line for a certain $P_{amb tol}$ and take 80% between the ambiant line and
the Buhlman tissue line.

So we can compute what would be $P_{tissue,tol}^{GF}$:

$P_{tissue,tol}^{GF} = P_{amb tol} + GF\left(P_{tissue,tol}-P_{amb tol}\right)$

This can be written as:

$P_{tissue,tol}^{GF} = P_{amb tol}\left( 1 + GF\left( \frac{1}{b}-1 \right) \right)+a \cdot GF$

Therefore:

$ceiling^{GF}= \frac{P_{tissue,tol}^{GF} - a \cdot GF}{1 + GF\left( \frac{1}{b}-1 \right)}$

We have an optional argument in the ceiling function to compute a ceiling with gradient factor:

In [1]:
from buhlmann import ceiling_pressure, Tissues, pressure_to_depth
initial_tissues = Tissues()
for gf_pc in list(range(0 , 110, 10)):
    print("GF={}, ceiling={}".format(
        gf_pc/100.0,
        pressure_to_depth(ceiling_pressure(initial_tissues, gf=gf_pc/100.0))))


GF=0.0, ceiling=-2.547929999999999
GF=0.1, ceiling=-2.806488783715432
GF=0.2, ceiling=-3.063201934707479
GF=0.3, ceiling=-3.3180891443154206
GF=0.4, ceiling=-3.5711698247513213
GF=0.5, ceiling=-3.8224631140283916
GF=0.6, ceiling=-4.071987880785298
GF=0.7, ceiling=-4.319762729008982
GF=0.8, ceiling=-4.565806002658449
GF=0.9, ceiling=-4.810135790191965
GF=1.0, ceiling=-5.052769928999998


Usually the diver defines two variables $GF_{lo}$ and $GF_{hi}$.

$GF_{lo}$ is used on the first stop.

$GF_{hi}$ is used on the last stop when surfacing.

Between these points, we interpolate linearly the $GF$.

To make it easier, we will just encapsulate this in a class.

In [2]:
class GradientFactors:
    def __init__(self, gf_lo, pamb_lo, gf_hi, pamb_hi=1):
        """
        :param gf_lo: gradient factor for first stop, usually between 0 and 1
        :param pamb_lo: ambiant pressure for the first stop
        :param gf_hi:  gradient factor for surfacing stop, usually between 0 and 1
        :param pamb_hi: ambiant pressure for the surface, assume 1 bar by default ?
        """
        self.gf_lo = gf_lo
        self.pamb_lo = pamb_lo
        self.gf_hi = gf_hi
        self.pamb_hi = pamb_hi

    def gf(self, pamb):
        return self.gf_lo + \
               (self.gf_hi - self.gf_lo) / (self.pamb_hi - self.pamb_lo) \
               * (pamb - self.pamb_lo)

gf = GradientFactors(0.5, 5, 0.8)
for p in range(1, 6):
    print("pressure={}, GF={}".format(p, gf.gf(p)))


pressure=1, GF=0.8
pressure=2, GF=0.7250000000000001
pressure=3, GF=0.65
pressure=4, GF=0.575
pressure=5, GF=0.5


we have a parameter to find stops with gradient factors


In [3]:
import pandas as pd
from buhlmann import Gas, Tissues, run_dive, get_stops_to_surface
gas = Gas(n2_pc=0.79, he_pc=0.0)
initial_tissues = Tissues()
dive_plan = pd.DataFrame([
    [0, 0],
    [2, 40],
    [22, 40]],
    columns=['t', 'depth'])
run_dive_plan = run_dive(dive_plan, initial_tissues, gas)
current_tissues = run_dive_plan.iloc[-1]['tissues']
current_depth = run_dive_plan.iloc[-1]['depth']
max_ascent_rate = 9  # metres/sec
run_stops = get_stops_to_surface(
    current_tissues,
    current_depth,
    gas,
    max_ascent_rate,
    gf_lo=0.3,
    gf_hi=0.8,
)
run_stops['t'] += run_dive_plan.iloc[-1]['t']
run_dive_plan = run_dive_plan.append(run_stops)

# print(run_dive_plan)
dive_data = run_dive(run_dive_plan, initial_tissues, gas, resolution=1)
# print(dive_data)

from matplotlib.patches import Polygon
import matplotlib.colors as mcolors

def gradient_fill(x, y, fill_color=None, ax=None, **kwargs):
    """
    Plot a line with a linear alpha gradient filled beneath it.

    Parameters
    ----------
    x, y : array-like
        The data values of the line.
    fill_color : a matplotlib color specifier (string, tuple) or None
        The color for the fill. If None, the color of the line will be used.
    ax : a matplotlib Axes instance
        The axes to plot on. If None, the current pyplot axes will be used.
    Additional arguments are passed on to matplotlib's ``plot`` function.

    Returns
    -------
    line : a Line2D instance
        The line plotted.
    im : an AxesImage instance
        The transparent gradient clipped to just the area beneath the curve.
    """
    if ax is None:
        ax = plt.gca()

    line, = ax.plot(x, y, **kwargs)
    if fill_color is None:
        fill_color = line.get_color()

    zorder = line.get_zorder()
    alpha = line.get_alpha()
    alpha = 1.0 if alpha is None else alpha

    z = np.empty((100, 1, 4), dtype=float)
    rgb = mcolors.colorConverter.to_rgb(fill_color)
    z[:,:,:3] = rgb
    z[:,:,-1] = np.linspace(0, alpha, 100)[:,None]

    # xmin, xmax, ymin, ymax = x.min(), x.max(), y.min(), y.max()
    xmin, xmax, ymin, ymax = x.min(), x.max(), 0, y.max()
    im = ax.imshow(z, aspect='auto', extent=[xmin, xmax, ymin, ymax],
                   origin='lower', zorder=zorder)

    xy = np.column_stack([x, y])
    xy = np.vstack([[xmin, ymin], xy, [xmax, ymin], [xmin, ymin]])
    clip_path = Polygon(xy, facecolor='none', edgecolor='none', closed=True)
    ax.add_patch(clip_path)
    im.set_clip_path(clip_path)

    ax.autoscale(True)
    return line, im

%matplotlib inline
import math
import numpy as np
from matplotlib import pyplot as plt
t = np.linspace(0, 2*math.pi, 800)
a = np.sin(t)
plt.figure(figsize=(12, 10), dpi=75)
# plt.plot(dive_data['t'], dive_data['depth'], 'r')
# plt.plot(dive_data['t'], dive_data['ceiling'], 'b')
fig, ax = plt.subplots(figsize=(12,10))
gradient_fill(dive_data['t'], dive_data['depth'], ax=ax)
gradient_fill(dive_data['t'], dive_data['ceiling'], ax=ax)
plt.gca().invert_yaxis()
# ax.set_aspect('equal')
ax.grid(True, which='both')

ax.axhline(y=0, color='k')
ax.axvline(x=0, color='k')
plt.show()

RuntimeError: looks like we cannot ascend after 100 minutes of stop